In [82]:
import pandas as pd
import numpy as np
import seaborn as sb
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

# import oscar and clean data

In [83]:
oscar = pd.read_csv("../data/the_oscar_award.csv")
oscar.category = oscar.category.str.lower()
filtered = oscar[oscar.category.str.contains("(actor)|(actress)|(directing)", regex = True)]
filtered.loc[filtered.category.str.contains("(actor)|(actress)"), 'is_director'] = 0
filtered.loc[filtered.category.str.contains("directing"), 'is_director'] = 1

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [84]:
oscar_ppl = filtered[~filtered.name.str.contains("( and)|(,)|(and )", regex = True)]
oscar_ppl.is_director = oscar_ppl.is_director.astype(np.uint8)
oscar_ppl.head()

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
0,1927,1928,1,actor,Richard Barthelmess,The Noose,False,0
1,1927,1928,1,actor,Emil Jannings,The Last Command,True,0
2,1927,1928,1,actress,Louise Dresser,A Ship Comes In,False,0
3,1927,1928,1,actress,Janet Gaynor,7th Heaven,True,0
4,1927,1928,1,actress,Gloria Swanson,Sadie Thompson,False,0


In [85]:
oscar_ppl.shape

(2192, 8)

In [86]:
filtered[filtered.name.str.contains("( and)|(,)|(and )", regex = True)]

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
531,1937,1938,10,actor in a supporting role,Roland Young,Topper,False,0.0
3768,1961,1962,34,directing,"Robert Wise, Jerome Robbins",West Side Story,True,1.0
5655,1978,1979,51,directing,"Warren Beatty, Buck Henry",Heaven Can Wait,False,1.0
5942,1981,1982,54,actor in a supporting role,"Howard E. Rollins, Jr.",Ragtime,False,0.0
6043,1982,1983,55,actor in a supporting role,"Louis Gossett, Jr.",An Officer and a Gentleman,True,0.0
6289,1984,1985,57,directing,Roland Joffé,The Killing Fields,False,1.0
6506,1986,1987,59,directing,Roland Joffé,The Mission,False,1.0
7558,1996,1997,69,actor in a supporting role,"Cuba Gooding, Jr.",Jerry Maguire,True,0.0
8832,2007,2008,80,directing,Joel Coen and Ethan Coen,No Country for Old Men,True,1.0
9184,2010,2011,83,directing,Joel Coen and Ethan Coen,True Grit,False,1.0


In [87]:
#Manually modify the rows where there are special characters (for movie after 1986)
oscar_ppl = pd.concat([oscar_ppl, filtered[(filtered.index==7558) | (filtered.index==6506)]])
oscar_ppl.shape

(2194, 8)

In [88]:
oscar_ppl[oscar_ppl.name=="Joel Coen and Ethan Coen"]

,year_film,year_ceremony,ceremony,category,name,film,winner,is_director


In [89]:
oscar_ppl = pd.concat([oscar_ppl, filtered[(filtered.index==8832) | (filtered.index==9184)]], ignore_index=False)
oscar_ppl = pd.concat([oscar_ppl, filtered[(filtered.index==8832) | (filtered.index==9184)]], ignore_index=True)
oscar_ppl.shape

(2198, 8)

In [90]:
oscar_ppl[oscar_ppl.name== "Joel Coen and Ethan Coen"]

,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
2194,2007,2008,80,directing,Joel Coen and Ethan Coen,No Country for Old Men,True,1.0
2195,2010,2011,83,directing,Joel Coen and Ethan Coen,True Grit,False,1.0
2196,2007,2008,80,directing,Joel Coen and Ethan Coen,No Country for Old Men,True,1.0
2197,2010,2011,83,directing,Joel Coen and Ethan Coen,True Grit,False,1.0


In [91]:
oscar_ppl.at[2194, 'name'] = 'Joel Coen'
oscar_ppl.at[2195, 'name'] = 'Ethan Coen'
oscar_ppl.at[2196, 'name'] = 'Ethan Coen'
oscar_ppl.at[2197, 'name'] = 'Joel Coen'

In [92]:
oscar_ppl.shape

(2198, 8)

In [93]:
oscar_ppl.head()

,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
0,1927,1928,1,actor,Richard Barthelmess,The Noose,False,0.0
1,1927,1928,1,actor,Emil Jannings,The Last Command,True,0.0
2,1927,1928,1,actress,Louise Dresser,A Ship Comes In,False,0.0
3,1927,1928,1,actress,Janet Gaynor,7th Heaven,True,0.0
4,1927,1928,1,actress,Gloria Swanson,Sadie Thompson,False,0.0


In [97]:
oscar_ppl[oscar_ppl.name=='Alejandro G. Iñárritu'].index
oscar_ppl.at[2062, 'name'] = 'Alejandro González Iñárritu'

In [98]:
oscar_ppl = oscar_ppl[oscar_ppl['year_film']>1985]
print('oscar award dataset has shape {}'.format(oscar_ppl.shape))
oscar_ppl.head()

oscar award dataset has shape (852, 8)


,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
1346,1986,1987,59,actor in a leading role,Dexter Gordon,'Round Midnight,False,0.0
1347,1986,1987,59,actor in a leading role,Bob Hoskins,Mona Lisa,False,0.0
1348,1986,1987,59,actor in a leading role,William Hurt,Children of a Lesser God,False,0.0
1349,1986,1987,59,actor in a leading role,Paul Newman,The Color of Money,True,0.0
1350,1986,1987,59,actor in a leading role,James Woods,Salvador,False,0.0


## movie industry

In [99]:
movie_industry = pd.read_csv("../data/movie_industry.csv", encoding = "ISO-8859-1" )
years = np.unique(movie_industry.year.values)
directors = np.unique(movie_industry.director.values)
actors = np.unique(movie_industry.star.values)
individuals = np.append(directors, actors)
years2 = np.repeat(years, len(individuals))
agents = np.tile(individuals, len(years))
is_director = np.tile(np.append(np.ones(len(directors)), np.zeros(len(actors))), len(years))
ID = np.tile(np.arange(len(individuals)), len(years))
time = np.repeat(np.arange(len(years)), len(individuals))


In [100]:
assert len(agents) == len(years2)
assert len(agents) == len(is_director)
assert len(agents) == len(ID)

In [101]:
df  = pd.DataFrame({"agent": agents,
                   "is_director": is_director,
                   "id": ID,
                   "year": years2,
                   "time": time})
print('There are {} people who double and actors and directors'.format(len(individuals)  - len(df.groupby(["agent"]).count())))
df

There are 171 people who double and actors and directors


,agent,is_director,id,year,time
0,A.R. Murugadoss,1.0,0,1986,0
1,Aamir Khan,1.0,1,1986,0
2,Aaron Blaise,1.0,2,1986,0
3,Aaron Harvey,1.0,3,1986,0
4,Aaron Kim Johnston,1.0,4,1986,0
...,...,...,...,...,...
163148,Zoey Deutch,0.0,5258,2016,30
163149,Zooey Deschanel,0.0,5259,2016,30
163150,Álex Angulo,0.0,5260,2016,30
163151,Élodie Bouchez,0.0,5261,2016,30


In [102]:
assert df.groupby("is_director").count().agent[1] == len(directors)*len(years)
assert len(df.groupby(["agent", "is_director"]).count()) == len(individuals)

In [103]:
%%time
total_nominations =[]
for i in df.index:
    if df.is_director[i]:
        temp = oscar_ppl[(oscar_ppl.name == df.agent[i]) & (oscar_ppl.is_director == True)]
        temp = temp[temp.year_film <= df.year[i]]
        nomination = temp.shape[0]
        total_nominations.append(nomination)
    else:
        temp = oscar_ppl[(oscar_ppl.name == df.agent[i]) & (oscar_ppl.is_director == False)]
        temp = temp[temp.year_film <= df.year[i]]
        nomination = temp.shape[0]
        total_nominations.append(nomination)

CPU times: user 3min 27s, sys: 1.3 s, total: 3min 29s
Wall time: 3min 30s


In [104]:
df["nominations"] = total_nominations
df.head()

,agent,is_director,id,year,time,nominations
0,A.R. Murugadoss,1.0,0,1986,0,0
1,Aamir Khan,1.0,1,1986,0,0
2,Aaron Blaise,1.0,2,1986,0,0
3,Aaron Harvey,1.0,3,1986,0,0
4,Aaron Kim Johnston,1.0,4,1986,0,0


In [ ]:
%%time
total_win_counts=[]
for i in df.index:
    if df.is_director[i]:
        temp = oscar_ppl[(oscar_ppl.name == df.agent[i]) & (oscar_ppl.is_director == True)]
        temp = temp[temp.year_film <= df.year[i]]
        win = sum(temp.winner.values)
        total_win_counts.append(win)
    else:
        temp = oscar_ppl[(oscar_ppl.name == df.agent[i]) & (oscar_ppl.is_director == False)]
        temp = temp[temp.year_film <= df.year[i]]
        win = sum(temp.winner.values)
        total_win_counts.append(win)

In [ ]:
df["win_counts"] = total_win_counts
df.head()

## sanity check

In [ ]:
df[df.agent == "Clint Eastwood"].head(50)

In [ ]:
oscar_ppl[oscar_ppl.name == "Clint Eastwood"].head(50)

In [35]:
individuals = set(individuals)

In [32]:
oscar_individuals = set(np.unique(oscar_ppl.name.values))

In [33]:
len(oscar_individuals)

488

In [37]:
len(individuals.intersection(oscar_individuals))

406

In [40]:
diff_n_ppl = oscar_individuals - individuals  
diff_n_ppl

{'Adriana Barraza',
 'Alec Guinness',
 'Alejandro G. Iñárritu',
 'Allison Janney',
 'Amy Ryan',
 'Ann Sothern',
 'Anne Archer',
 'Anne Ramsey',
 'Armin Mueller-Stahl',
 'Barkhad Abdi',
 'Bong Joon Ho',
 'Brenda Fricker',
 'Bruce Davison',
 'Chazz Palminteri',
 'Cuba Gooding, Jr.',
 'Cynthia Erivo',
 'Daniel Day Lewis',
 'Daniel Kaluuya',
 'David Paymer',
 'Dean Stockwell',
 'Demián Bichir',
 'Denholm Elliott',
 'Dianne Wiest',
 'Emmanuelle Riva',
 'Gerard Depardieu',
 'Gloria Stuart',
 'Graham Greene',
 'Hal Holbrook',
 'J.K. Simmons',
 'Jacki Weaver',
 'James Coburn',
 'Jaye Davidson',
 'Jennifer Hudson',
 'Jessica Tandy',
 'Joan Cusack',
 'Joan Plowright',
 'Jonathan Pryce',
 'Jordan Peele',
 'June Squibb',
 'Kate Nelligan',
 'Kathleen Quinlan',
 'Lady Gaga',
 'Lauren Bacall',
 'Laurie Metcalf',
 'Lena Olin',
 'Lesley Manville',
 'Lucas Hedges',
 "Lupita Nyong'o",
 'Lynn Redgrave',
 'Mare Winningham',
 'Margot Robbie',
 'Marianne Jean-Baptiste',
 'Marina de Tavira',
 'Marlee Matlin',

In [79]:
oscar[oscar.name == 'Alejandro G. Iñárritu']

,year_film,year_ceremony,ceremony,category,name,film,winner
9675,2014,2015,87,directing,Alejandro G. Iñárritu,Birdman or (The Unexpected Virtue of Ignorance),True
9802,2015,2016,88,directing,Alejandro G. Iñárritu,The Revenant,True


In [81]:
movie_industry[movie_industry['director'] == 'Alejandro González Iñárritu']

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
3121,2000000.0,Altavista Films,Mexico,Alejandro González Iñárritu,Drama,5383834.0,Amores Perros,R,2001-04-13,154,8.1,Emilio Echevarría,187814,Guillermo Arriaga,2000
3793,20000000.0,This Is That Productions,USA,Alejandro González Iñárritu,Crime,16248701.0,21 Grams,R,2004-01-16,124,7.7,Sean Penn,200427,Guillermo Arriaga,2003
4429,25000000.0,Paramount Pictures,France,Alejandro González Iñárritu,Drama,34302837.0,Babel,R,2006-11-10,143,7.5,Brad Pitt,257380,Guillermo Arriaga,2006
5380,0.0,Menageatroz,Mexico,Alejandro González Iñárritu,Drama,5101237.0,Biutiful,R,2011-02-04,148,7.5,Javier Bardem,74415,Alejandro González Iñárritu,2010
6180,18000000.0,New Regency Pictures,USA,Alejandro González Iñárritu,Comedy,42340598.0,Birdman or (The Unexpected Virtue of Ignorance),R,2014-11-14,119,7.8,Michael Keaton,455260,Alejandro González Iñárritu,2014
6387,135000000.0,Regency Enterprises,Hong Kong,Alejandro González Iñárritu,Adventure,183637894.0,Revenant: El Renacido,R,2016-01-08,156,8.0,Leonardo DiCaprio,522384,Mark L. Smith,2015


In [75]:
film_names = []
xdirectos = []
xstars = []
target_ppls = []


for ppl in diff_n_ppl: 
    film_name = oscar[oscar.name == ppl].film.values.tolist()
    for f in film_name:
        temp = movie_industry[movie_industry['name'] == f]
        film_names.append(f)
        xdirectos.append(temp.director.values.tolist())
        xstars.append(temp.star.values.tolist())
        target_ppls.append(ppl)
      

In [72]:
oscar[oscar.name == 'Amy Ryan'].film.values.tolist()[0]

'Gone Baby Gone'

In [78]:
df  = pd.DataFrame({"film": film_names,
                   "x_directors": xdirectos,
                   "x_stars":xstars ,
                   "target_ppls": target_ppls})

pd.set_option("display.max_rows", 102)
df

,film,x_directors,x_stars,target_ppls
0,Georgia,[Ulu Grosbard],[Jennifer Jason Leigh],Mare Winningham
1,Parasite,[],[],Bong Joon Ho
2,Birdman or (The Unexpected Virtue of Ignorance),[Alejandro González Iñárritu],[Michael Keaton],Alejandro G. Iñárritu
3,The Revenant,[],[],Alejandro G. Iñárritu
4,The Crying Game,[Neil Jordan],[Stephen Rea],Jaye Davidson
5,Whiplash,[Damien Chazelle],[Miles Teller],J.K. Simmons
6,Tom & Viv,[Brian Gilbert],[Willem Dafoe],Rosemary Harris
7,A Room with a View,[],[],Denholm Elliott
8,Frozen River,[],[],Melissa Leo
9,The Fighter,[David O. Russell],[Mark Wahlberg],Melissa Leo
